## Pseudo Random Number Generators in PyTorch
Key takeaways from last lecture:
* random numbers in your code are never truly random!
* think about where your pseudo random numbers come from and how you can control their generation!

### Default random number generator in Python
It's the Mersenne Twister!
* just don't use the distributions, use the numpy equivalent instead (e.g., `_rng.choice(...)`, `_rng.shuffle(_arr)`)
* functions for sequences: `random.choice`, `random.shuffle`, `random.sample`
    * use  the equivalent numpy functions or draw a random index using it
* still, if you *need* reproducible code, you need to seed this RNG

In [ ]:
import random
import secrets

_my_python_seed = secrets.randbits(128)  # save this seed for reproducibilty
random.seed(_my_python_seed)

Caveats from the [python docs](https://docs.python.org/3/library/random.html):
> By reusing a seed value, the same sequence should be reproducible from run to run as long as multiple threads are not running.

> Most of the random module’s algorithms and seeding functions are **subject to change** across Python versions [...]

### Default random number generator in PyTorch
It's the Mersenne Twister! (I kid you not)

### Seeding pseudo random number generators in PyTorch
There is a convenience function provided by torch to seed its global random number generators.

In [ ]:
import torch

# seed the Mersenne Twister on the CPU and Philox generator(s) on the GPU(s)
torch.manual_seed(1717800298)

You may find in old documentation or other libraries' use of PyTorch the following additional statements. However, `torch.manual_seed()` will seed the generators on the GPU(s) as well.
```
# seed the generator on a single GPU
torch.cuda.manual_seed(_my_manual_single_cuda_seed)

# or seed the different generators on multiple GPUs
torch.cuda.manual_seed_all(_my_manual_multi_cuda_seed)
```

PyTorch uses CUDA to run on nvida graphics cards. The separate pseudo RNGs there use the `Philox_4x32_10` generator.
The [documentation](https://docs.nvidia.com/cuda/curand/device-api-overview.html#bit-generation-3) of the `curand` package (part of nvidia's CUDA toolkit) recommends:
> For the highest quality parallel pseudorandom number generation, each experiment should be assigned a unique seed value. Within an experiment, each thread of computation should be assigned a unique id number.

For our training script, we create a convenience function that takes a seed and makes PyTorch run *as deterministic as possible*. 

In [ ]:
# src.utils

import numpy as np
import torch
import random

def seed_torch(seed, reproducible:bool = False):
    np.random.seed(seed % (2**32-1))  # numpy global rng 
    torch.manual_seed(seed % (2**32-1))  # torch global rng (CPU & CUDA)
    random.seed(seed % (2**32-1))  # python global rng (MT)

    if reproducible:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

We also have to seed the dataloaders or provide them an instance of `torch.Generator`. The [PyTorch docs](https://pytorch.org/docs/stable/notes/randomness.html) recommend the following code. We are going to refrain from using worker threads for the dataloader, so we are only going to pass a generator instance later.

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    worker_init_fn=seed_worker,
    generator=g,
)

### Application: Randomly initializing weights
Before we start training our neural network, all its weights (& biases) have to be initialized. This initialization can have a significant impact on the final performance of the trained network. PyTorch automatically performs standard initialization with random numbers. There is some research into more sophisticated initialization strategies (about the distribution to draw from), depending on the network architecture, but we are going to stick with PyTorch's standard initialization scheme.

The only problem? This initialization function currently *does not support* a `generator` keyword. This means, the random numbers come from PyTorch's global RNG. So we are going to *reimplement* this function. Our only change is to allow passing through a `generator` argument to the unform sampling function. 

In [ ]:
# src.utils

import warnings
import math
import torch
from torch.nn.init import _calculate_correct_fan, calculate_gain

def kaiming_uniform_(
    tensor: torch.Tensor, a: float = 0, mode: str = 'fan_in', nonlinearity: str = 'leaky_relu', generator: torch.Generator | None = None
):
    """Reimplementation of torch.nn.init.kaiming_uniform_ with generator arg"""
    if torch.overrides.has_torch_function_variadic(tensor):
        return torch.overrides.handle_torch_function(
            kaiming_uniform_,
            (tensor,),
            tensor=tensor,
            a=a,
            mode=mode,
            nonlinearity=nonlinearity)

    if 0 in tensor.shape:
        warnings.warn("Initializing zero-element tensors is a no-op")
        return tensor
    fan = _calculate_correct_fan(tensor, mode)
    gain = calculate_gain(nonlinearity, a)
    std = gain / math.sqrt(fan)
    bound = math.sqrt(3.0) * std  # Calculate uniform bounds from standard deviation
    with torch.no_grad():
        return tensor.uniform_(-bound, bound, generator=generator)

### Application: randomly splitting your data into train and validation splits
It is common practice to create a hold-out set for monitoring the training process for signs of overfitting and potentially selecting parameters on a trained network. This hold-out set is called the validation set and typically created by randomly splitting the whole dataset available for training into one smaller subset used for training and another as the hold-out subset. 

In [ ]:
from torch.utils.data import random_split

_split_gen = torch.Generator()
_split_gen.manual_seed(2080986636)

_train_data, _val_data = random_split(range(10), lengths=[0.8, 0.2], generator=_split_gen)
_train_data.indices, _val_data.indices

For our training today, we are going to need to fix the global seeds and create one local `torch.Generator` instance to pass around:

In [ ]:
# create random seeds
import secrets

_global_seed = secrets.randbits(128) % (2**32-1)
_local_seed = secrets.randbits(128) % (2**32-1)

In [ ]:
# these would also have to be tracked in your tool
print(f"Set all global seeds to: {_global_seed}")
print(f"Set the local torch seed to {_local_seed}")

# seed our generators for reproducibility
seed_torch(_global_seed, reproducible=True)
_rng_cpu = torch.Generator()
_rng_cpu.manual_seed(_local_seed)

# Training a neural network in PyTorch
Selecting a suitable model architecture depends on the type of data we have and the task it should learn. 

Today, we are going to train an image classifier on the CIFAR10 dataset.

## The dataset
The CIFAR10 dataset consists of 60,000 RGB color images (32x32) in 10 classes:  `plane`, `car`, `bird`, `cat`, `deer`, `dog`, `frog`, `horse`, `ship`, `truck`. PyTorch conveniently allows us to directly download this dataset using `torchvision.datasets.CIFAR10`.

It is already split into 50,000 training and 10,000 testing samples. As the convenience function directly creates a pytorch `dataset` object, we need to provide the call with the transformations that we would like to apply. In this case, we merely need to normalize the data and turn them into tensors.

In [ ]:
import torchvision.transforms as transforms

# calculated beforehand
_cifar_means = (0.4914, 0.4822, 0.4465)
_cifar_stds = (0.2023, 0.1994, 0.2010)

_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(_cifar_means, _cifar_stds)
])

In [ ]:
# this cell will take a while on first executing it, because the dataset is downloaded in the background

import torchvision

_train_data_full = torchvision.datasets.CIFAR10(root="../data", train=True, download=True, transform=_transform)

_test_data_full = torchvision.datasets.CIFAR10(root="../data", train=False, download=True, transform=_transform)

## The model
Next, we need a model. We are going to use a simplified version of the well-known `ResNet` family. This architecture uses convolutions and pooling operations in different blocks. At the end, it has a fully-connected layer for the classification. Our version `ResNet9` uses the modified weight initalization that we have introduced above.

In [ ]:
# src.models

import torch.nn as nn

def conv_bn(channels_in, channels_out, kernel_size=3, stride=1, padding='same', bn=True, activation=True):
    op = [
            nn.Conv2d(channels_in, channels_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
    ]
    if bn:
        op.append(nn.BatchNorm2d(channels_out))
    if activation:
        op.append(nn.ReLU(inplace=True))
    return nn.Sequential(*op)


class Residual(nn.Module):
    def __init__(self, module):
        super(Residual, self).__init__()
        self.module = module

    def forward(self, x):
        return x + self.module(x)

In [ ]:
class ResNet9(nn.Module):
    def __init__(self, rng:torch.Generator, num_class:int = 10, **kwargs):
        super(ResNet9, self).__init__()

        self.network = nn.Sequential(
            conv_bn(3, 64, kernel_size=3, stride=1, padding='same'),
            nn.MaxPool2d(2),

            conv_bn(64, 128, kernel_size=3, stride=1, padding='same'),
            nn.MaxPool2d(2),
            Residual(nn.Sequential(
                conv_bn(128, 128),
                conv_bn(128, 128),
            )),

            conv_bn(128, 256, kernel_size=3, stride=1, padding='same'),
            nn.MaxPool2d(2),

            conv_bn(256, 256, kernel_size=3, stride=1, padding='same'),
            nn.MaxPool2d(2),
            Residual(nn.Sequential(
                conv_bn(256, 256),
                conv_bn(256, 256),
            )),

            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, num_class, bias=False),
        )

        self.reset_parameters(rng)

    def reset_parameters(self, rng: torch.Generator):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                #nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                kaiming_uniform_(m.weight, nonlinearity="relu", generator=rng)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                kaiming_uniform_(m.weight, nonlinearity="linear", generator=rng)
                # we have disabled the bias term in the linear layers, no need to init

    def forward(self, x):
        return self.network(x)

In [ ]:
_model = ResNet9(rng=_rng_cpu, num_class=10)

## The training
To train our model, we need an optimizer, that will modify the weights according to our models previous performance. We are going to use simple *stochastic gradient descent*. It needs three hyperparameters: learning rate, momentum and weight decay rate.

In [ ]:
import torch.optim as optim

_learning_rate = 1e-3
_momentum = 0.9
_weight_decay = 1e-3
_optimizer = optim.SGD(_model.parameters(), lr=_learning_rate, momentum=_momentum, nesterov=True, weight_decay=_weight_decay)

Further, we need to measure how well the network is doing at its task. We are going to use `CrossEntropyLoss` to compare class predictions with the true class labels.

In [ ]:
_criterion = nn.CrossEntropyLoss()

Now, onto the training! We define a function `fit` that takes our training data and trains our model `epoch_num` many times on it. This is also where we can move our computations to a GPU, if we have one.

In [ ]:
# src.train

from torch.utils.data import DataLoader

def fit(data, model, optimizer, criterion, epoch_num:int, batch_size:int, device, dl_rng):

    model = model.to(device)

    _is_cuda = device.type == 'cuda'

    _train_loader = DataLoader(dataset=data, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=0, pin_memory=_is_cuda, generator=dl_rng)
    
    _epoch_loss_train = []

    model.train()

    for _epoch in range(epoch_num):
        _loss = 0
        for _inputs, _labels in _train_loader:
            _inputs = _inputs.to(device)
            _labels = _labels.to(device)
            
            optimizer.zero_grad()

            _outputs = model(_inputs)

            _train_loss = criterion(_outputs, _labels)
            _train_loss.backward()

            optimizer.step()

            _loss += _train_loss.item()

        _loss /= len(_train_loader)

        print(f"Epoch {_epoch+1}/{epoch_num}, average train_loss: {_loss:.4f}")
        _epoch_loss_train.append(_loss)

    return _epoch_loss_train

Any trained neural network typically performs best on the data that it was trained on. To check how well it would do with new data, we are also going to run our model on separate data from the same distribution. This is our *test* set. We define another function for this purpose, which we call `predict`:

In [ ]:
def predict(data, model, criterion, batch_size:int, device, dl_rng):
    
    model = model.to(device)
    
    _is_cuda = device.type == 'cuda'
    
    _loader = DataLoader(dataset=data, batch_size=batch_size, drop_last=False, shuffle=False, num_workers=0, pin_memory=_is_cuda, generator=dl_rng)
    
    _preds = []
    _pred_errs = []
    
    model.eval()

    for _inputs, _labels in _loader:
        _inputs = _inputs.to(device)
        _labels = _labels.to(device)

        _outputs = model(_inputs)

        _pred_loss = criterion(_outputs, _labels)

        _preds.append(_outputs.softmax(dim=1))
        _pred_errs.append(_pred_loss)

    return torch.cat(_preds), torch.cat(_pred_errs)

Time to try this out!

In [ ]:
_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define the last hyperparameters
_epoch_num = 22
_batch_size = 500

# train
_epoch_loss = fit(_train_data_full, _model, _optimizer, _criterion, _epoch_num, _batch_size, _device, _rng_cpu)

# test
_test_criterion = nn.CrossEntropyLoss(reduction='none')

_preds, _pred_errs = predict(_test_data_full, _model, _test_criterion, _batch_size, _device, _rng_cpu)


print(f"\nEpoch {_epoch_num}/{_epoch_num}, average test_loss: {_pred_errs.sum()/(len(_test_data_full)):.4f}")

In [ ]:
# save the trained model for later (optional)
#torch.save(_model.state_dict(), "./models/CIFAR10_22.pth")

In [ ]:
import plotly.express as px

px.line(_epoch_loss)

To obtain the predicted classes, we have to find the index of the highest value in the predictions vector.

In [ ]:
# reload our model
_rng_model = torch.Generator()
_rng_model.manual_seed(4072885555)
_saved_net = ResNet9(rng=_rng_model)
_saved_net.load_state_dict("./models/CIFAR10_22.pth")